# Inport needed package

- import os, sys # to add the parent directory to the path

In [1]:
import os
import sys
import time

- Using torchvision to create a dataset

In [2]:
# Using torchvision to create a dataset
import cv2
from torchvision import transforms
import torch
from torch.utils.data import random_split, DataLoader
import torchvision

import pandas as pd

- import self library

In [3]:
from train.trainer import ClassifierTrainer as Trainer
import dataset as ds  # type: ignore
import model as md  # type: ignore

# Define classification train process

1. Define place where the model is saved

In [4]:
time_str = time.strftime("%Y%m%d_%H%M%S")

2. Define train function

In [5]:
def doTheTrain(dataset, model):
  # define batch_size
  batch_size = 64

  # init train val test ds
  train_val_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_val_size
  train_ds, test_ds = random_split(dataset, [train_val_size, test_size])

  # define optimizer using Adam and loss function
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  trainer = Trainer(model, optimizer, loss_fn, random_seed_value=86)
  print('device: ', trainer.device)
  avg_loss, metric = trainer.cross_validate(train_ds, k=5, epochs=10, batch_size=batch_size)
  print('avg_loss: ', avg_loss)

  # score model
  test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
  model_scored = trainer.score(test_dataloader)
  print(f'model_scored: {model_scored:.4f}, avg_accuracy: {100*(1 - model_scored):.4f}')

  # return model scored, train_avg_lost
  return model_scored, avg_loss

3. execute progress

- define the model

In [6]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

- Define tested datasets

In [7]:
datasets = {
    'gi4e_full': ds.Gi4eDataset(
        './datasets/gi4e',
        transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor()]),
        is_classification=True),
    'gi4e_raw_eyes': ds.ImageDataset(
        './datasets/gi4e_raw_eyes',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
    'gi4e_detected_eyes': ds.ImageDataset(
        './datasets/gi4e_eyes/20250521_200316',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
}

- Train all defined model on each registered datasets

In [8]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for name, dataset in datasets.items():
	for model in models:
		print(f'Running {name} dataset with {model.__class__.__name__}')
		# do the train
		start_time = time.time()
		scored, loss = doTheTrain(dataset, model)
		end_time = time.time()
		total_time = end_time - start_time
		print(f'Finished {name} dataset with {model.__class__.__name__}')
		print('----------------------')

		# save the result
		result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model.__class__.__name__],
			'dataset': [name],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
		})], ignore_index=True)

print('Finished all datasets')

# print the result
print(result_df)



Running gi4e_full dataset with ResNet
device:  cuda
Fold 1/5:
Epoch 1/10, Train Loss: 2.3362, Test Loss: 0.4823, Total Time: 00 hours, 00 minutes, 24 seconds
Epoch 2/10, Train Loss: 0.0202, Test Loss: 0.0091, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 3/10, Train Loss: 0.0013, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 4/10, Train Loss: 0.0003, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 5/10, Train Loss: 0.0002, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 6/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 7/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 8/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 9/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 10/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 

C:\Users\nnvuf\AppData\Local\Temp\ipykernel_28364\3706550783.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({


Epoch 1/10, Train Loss: 2.0503, Test Loss: 2.0258, Total Time: 00 hours, 00 minutes, 24 seconds
Epoch 2/10, Train Loss: 0.0292, Test Loss: 0.0757, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 3/10, Train Loss: 0.0050, Test Loss: 0.0045, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 4/10, Train Loss: 0.0018, Test Loss: 0.0013, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 5/10, Train Loss: 0.0009, Test Loss: 0.0008, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 6/10, Train Loss: 0.0006, Test Loss: 0.0006, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 7/10, Train Loss: 0.0005, Test Loss: 0.0005, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 8/10, Train Loss: 0.0003, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 9/10, Train Loss: 0.0003, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 10/10, Train Loss: 0.0002, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 17 seconds
Fold 1/5, Total Test Loss: 0.0003, Fold

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 1/5, Total Test Loss: 0.1197, Fold accuracy: 88.0342
Fold 2/5:
Epoch 1/10, Train Loss: 0.1992, Test Loss: 0.0240, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 2/10, Train Loss: 0.1017, Test Loss: 0.0180, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 3/10, Train Loss: 0.1340, Test Loss: 0.0392, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 4/10, Train Loss: 0.1911, Test Loss: 0.0413, Total Time: 00 hours, 00 minutes, 22 seconds
Epoch 5/10, Train Loss: 0.0867, Test Loss: 0.0011, Total Time: 00 hours, 00 minutes, 28 seconds
Epoch 6/10, Train Loss: 0.0436, Test Loss: 0.0079, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 7/10, Train Loss: 0.0568, Test Loss: 0.0022, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 8/10, Train Loss: 0.0312, Test Loss: 0.0072, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 9/10, Train Loss: 0.0217, Test Loss: 0.0091, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 10/10, Train Loss: 0.0873, Test Loss: 0.0020, Total Time: 00 h

...

Fold 3/5, Total Test Loss: 0.9879, Fold accuracy: 82.7681
Fold 4/5:
Epoch 1/10, Train Loss: 0.0353, Test Loss: 0.0695, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 2/10, Train Loss: 0.0395, Test Loss: 0.0405, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 3/10, Train Loss: 0.0222, Test Loss: 0.0145, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 4/10, Train Loss: 0.0116, Test Loss: 0.0081, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 5/10, Train Loss: 0.0020, Test Loss: 0.0041, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 6/10, Train Loss: 0.0119, Test Loss: 0.0583, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 7/10, Train Loss: 0.0142, Test Loss: 0.0406, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 8/10, Train Loss: 0.0074, Test Loss: 0.0022, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 9/10, Train Loss: 0.0005, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 10/10, Train Loss: 0.0003, Test Loss: 0.0002, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 3/5, Total Test Loss: 0.8479, Fold accuracy: 81.3369
Fold 4/5:
Epoch 1/10, Train Loss: 0.1529, Test Loss: 0.0261, Total Time: 00 hours, 00 minutes, 23 seconds
Epoch 2/10, Train Loss: 0.1323, Test Loss: 0.0409, Total Time: 00 hours, 00 minutes, 23 seconds
Epoch 3/10, Train Loss: 0.1591, Test Loss: 0.0661, Total Time: 00 hours, 00 minutes, 23 seconds
Epoch 4/10, Train Loss: 0.1887, Test Loss: 0.0310, Total Time: 00 hours, 00 minutes, 22 seconds
Epoch 5/10, Train Loss: 0.1138, Test Loss: 0.0568, Total Time: 00 hours, 00 minutes, 23 seconds
Epoch 6/10, Train Loss: 0.0590, Test Loss: 0.0099, Total Time: 00 hours, 00 minutes, 24 seconds
Epoch 7/10, Train Loss: 0.0713, Test Loss: 0.0184, Total Time: 00 hours, 00 minutes, 23 seconds
Epoch 8/10, Train Loss: 0.0631, Test Loss: 0.0253, Total Time: 00 hours, 00 minutes, 24 seconds
Epoch 9/10, Train Loss: 0.0744, Test Loss: 0.0527, Total Time: 00 hours, 00 minutes, 24 seconds
Epoch 10/10, Train Loss: 0.1013, Test Loss: 0.0342, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 1/5, Total Test Loss: 0.5704, Fold accuracy: 42.9605
Fold 2/5:
Epoch 1/10, Train Loss: 0.0329, Test Loss: 0.2599, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 2/10, Train Loss: 0.0451, Test Loss: 0.7276, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 3/10, Train Loss: 0.0125, Test Loss: 0.0475, Total Time: 00 hours, 00 minutes, 13 seconds
Epoch 4/10, Train Loss: 0.0076, Test Loss: 0.0236, Total Time: 00 hours, 00 minutes, 14 seconds
Epoch 5/10, Train Loss: 0.0014, Test Loss: 0.0226, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 6/10, Train Loss: 0.0008, Test Loss: 0.0024, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 7/10, Train Loss: 0.0002, Test Loss: 0.0017, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 8/10, Train Loss: 0.0002, Test Loss: 0.0017, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 9/10, Train Loss: 0.0002, Test Loss: 0.0016, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 10/10, Train Loss: 0.0001, Test Loss: 0.0018, Total Time: 00 h

4. print the result

In [9]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)

              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000005     99.999365   
1           gi4e_full  DenseNet  0.000087     99.994973   
2           gi4e_full       VGG  0.053988     88.219118   
3       gi4e_raw_eyes    ResNet  0.197609     94.068484   
4       gi4e_raw_eyes  DenseNet  0.014683     98.544904   
5       gi4e_raw_eyes       VGG  0.193054     60.202415   
6  gi4e_detected_eyes    ResNet  0.001794     99.958043   
7  gi4e_detected_eyes  DenseNet  0.114782     99.552733   
8  gi4e_detected_eyes       VGG  0.045642     84.694791   

                 total_time  
0 0 days 00:20:56.201085806  
1 0 days 00:15:49.252779484  
2 0 days 00:19:33.275934696  
3 0 days 00:09:33.807798624  
4 0 days 00:13:07.467339516  
5 0 days 00:21:28.160236597  
6 0 days 00:14:06.130110501  
7 0 days 00:13:07.358267069  
8 0 days 00:15:25.128678799  


In [10]:
print(result_df)

              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000005     99.999365   
1           gi4e_full  DenseNet  0.000087     99.994973   
2           gi4e_full       VGG  0.053988     88.219118   
3       gi4e_raw_eyes    ResNet  0.197609     94.068484   
4       gi4e_raw_eyes  DenseNet  0.014683     98.544904   
5       gi4e_raw_eyes       VGG  0.193054     60.202415   
6  gi4e_detected_eyes    ResNet  0.001794     99.958043   
7  gi4e_detected_eyes  DenseNet  0.114782     99.552733   
8  gi4e_detected_eyes       VGG  0.045642     84.694791   

                 total_time  
0 0 days 00:20:56.201085806  
1 0 days 00:15:49.252779484  
2 0 days 00:19:33.275934696  
3 0 days 00:09:33.807798624  
4 0 days 00:13:07.467339516  
5 0 days 00:21:28.160236597  
6 0 days 00:14:06.130110501  
7 0 days 00:13:07.358267069  
8 0 days 00:15:25.128678799  


# Embedded Classification

- Define the model

In [11]:
embedded_models = [md.FeatureExtractor(model) for model in models]
classifier_models = [md.Classifier(model) for model in models]


TypeError: 'ResNet' object is not subscriptable

- Load the trained weight to the model

In [ ]:
# ignore

- Transform the dataset to the embedded data

In [ ]:
classifier_df = pd.DataFrame(columns=['key', 'dataset', 'model'])

for name, dataset in datasets.items():
  for model in embedded_models:
    print(f'Getting features for {name} dataset with {model.__class__.__name__}')

    # get the classifier model from the model
    classifier_model = next((m for m in classifier_models if m.__class__.__name__ ==
                            model.__class__.__name__.replace('FeatureExtractor', 'Classifier')), None)
    if classifier_model is None:
      print(f'Classifier model not found for {model.__class__.__name__}')
      continue

    model_dataset = ds.EmbeddedDataset(dataset, model)

    classifier_df = pd.concat([classifier_df, pd.DataFrame({
        'key': [f'{name}_{model.__class__.__name__}'],
        'model': [classifier_model],
        'dataset': [model_dataset]
    })], ignore_index=True)

    print(f'Finished getting features for {name} dataset with {model.__class__.__name__}')

Getting features for gi4e_full dataset with FeatureExtractor(VGG)


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

- Train all defined model on each registered datasets

In [ ]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for index, row in classifier_df.iterrows():
  dataset = row['dataset']
  model = row['model']
  key = row['key']

  print(f'Running {key} dataset with {model.__class__.__name__}')
  # do the train
  start_time = time.time()
  scored, loss = doTheTrain(dataset, model)
  end_time = time.time()
  total_time = end_time - start_time
  print(f'Finished {key} dataset with {model.__class__.__name__}')
  print('----------------------')

  # save the result
  result_df = pd.concat([result_df, pd.DataFrame({
      'model': [model.__class__.__name__],
      'dataset': [dataset.__class__.__name__],
      'avg_loss': [loss],
      'avg_accuracy': [scored],
      'total_time': [total_time]
  })], ignore_index=True)

Running gi4e_full dataset with Classifier


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


- Print the result

In [ ]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)

In [ ]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

for model in models:
  # explore the model
	print(f'Model: {model.__class__.__name__}')
	
	# explain the model
	print(f'Number of layers: {len(list(model.children()))}')
	print(model)

Model: ResNet
Number of layers: 10
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 2